# FUNCTION

In [3]:
import torch
import random
import torchvision
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch.nn as nn
import tool
%matplotlib inline
import importlib
importlib.reload(tool)

<module 'tool' from '/Users/jacky/Documents/code/my3/one/tool.py'>

In [ ]:
class mnist_data(data.Dataset):
    def __init__(self,x,y):
        self.x=x.float()
        self.y=y.float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        if self.y is not None:
            return self.x[index].reshape([-1]),self.y[index]
        else:
            return self.x[index].reshape([-1])


In [ ]:
class Auto_Learning(nn.Module):
    def __init__(self,input_a,hidden1_a,hidden2_a,hidden3_a,rand_seed=10,device=torch.device('cpu')):
        torch.manual_seed(rand_seed)
        super(Auto_Learning,self).__init__()
        input_a=input_a*input_a
        hidden1_a*=hidden1_a
        hidden2_a*=hidden2_a
        hidden3_a*=hidden3_a
        self.device=device

        self.net=nn.Sequential(
            nn.Linear(input_a,hidden1_a),
            nn.ReLU(),
            nn.Linear(hidden1_a,hidden2_a),
            nn.ReLU(),
            nn.Linear(hidden2_a,hidden3_a),
            nn.ReLU(),
            nn.Linear(hidden3_a,hidden2_a),
            nn.ReLU(),
            nn.Linear(hidden2_a,hidden1_a),
            nn.ReLU(),
            nn.Linear(hidden1_a,input_a),
            nn.ReLU()
        )
        self.lossF=nn.MSELoss()
    def forward(self,input):
        input=input.to(self.device)
        return self.net(input)
    
    def trainF(self,train_data,valid_data,learning_rate=0.1,epoch=100,epoch_per=10,batch_size=128,optim=None):
        lossF=self.lossF
        if optim is None:
            optim=torch.optim.Adam(self.parameters(),lr=learning_rate)
        loader=data.DataLoader(train_data,batch_size=batch_size)
        train_record=[]
        valid_record=[]

        for i in range(epoch):
            train_loss=0.0
            pb=tool.ProgressBar(epoch_per*len(train_data))
            for k in range(epoch_per):
                for x,_ in loader:
                    x=x.to(device)
                    optim.zero_grad()
                    res=self(x)
                    loss=lossF(res,x)
                    loss.backward()
                    optim.step()
                    train_loss+=loss.tolist()*len(x)
                    pb.step(len(x))
                    pass
            #compute
            pb.end()
            train_loss/=len(train_data)
            train_loss/=epoch_per
            train_record.append(train_loss)
            valid_loss=self.valid(valid_data)
            valid_record.append(valid_loss)
            #print
            print(f'{i}\t{train_loss:.4f}\t{valid_loss:.4f}')
    
        return train_record,valid_record
    def valid(self,valid_data,batch_size=128):
        lossF=self.lossF
        loader=data.DataLoader(valid_data)
        total_loss=0.0
        for x,_ in loader:
            x=x.to(self.device)
            res=self(x)
            loss=lossF(res,x).tolist()
            total_loss+=loss*len(x)
        return total_loss/len(valid_data)
    def __half_farward__(self,input):
        input=input.to(self.device)
        net2=nn.Sequential(self.net[0:6])
        res=net2(input)
        return res
    def get_code(self,input):
        res=[]
        loader=data.DataLoader(input,batch_size=1)
        for x,_ in input:
            hidden=self.__half_farward__(x)
            res.append(hidden.tolist())
        return res



# DATA

In [ ]:
# mnist=torchvision.datasets.MNIST('./mnist',download=True)
mnist=torchvision.datasets.MNIST('./mnist')

In [ ]:
train_x,test_x=tool.data_split(mnist.train_data,rand=20)
train_y,test_y=tool.data_split(mnist.train_labels,rand=20)
train_x,valid_x=tool.data_split(train_x,rand=10)
train_y,valid_y=tool.data_split(train_y,rand=10)

In [ ]:
train_data=mnist_data(train_x,train_y)
valid_data=mnist_data(valid_x,valid_y)
test_data=mnist_data(test_x,test_y)

In [ ]:
train_data[0][0].shape

In [ ]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

# AUTO_LEARNING

## train

In [ ]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
# ALmodel=Auto_Learning(28,30,20,10,device=device).to(device)
# print(ALmodel)
ALmodel=torch.load('./data/al_model').to(device)

In [ ]:
tr,vr=ALmodel.trainF(train_data,valid_data,learning_rate=0.001,batch_size=10,epoch_per=1)

In [ ]:
a=100
plt.matshow(ALmodel(train_data[a][0].reshape([-1])).reshape([28,28]).detach().cpu())
plt.matshow(train_data[a][0].reshape([28,28]))

## get code

In [ ]:
ALmodel=torch.load('./data/al_model')

In [ ]:
train_hidden=ALmodel.get_code(train_data)
valid_hidden=ALmodel.get_code(valid_data)
test_hidden=ALmodel.get_code(test_data)

In [ ]:
train_data2=mnist_data(torch.tensor(train_hidden),train_y.long())
test_data2=mnist_data(torch.tensor(test_hidden),test_y.long())
valid_data2=mnist_data(torch.tensor(valid_hidden),valid_y.long())

In [ ]:
torch.save(train_data2,'./data/train_hidden')
torch.save(valid_data2,'./data/valid_hidden')
torch.save(test_data2,'./data/test_hidden')